In [ ]:
import numpy as np

In [ ]:
import networkx as nx

In [ ]:
class ClusterState:

    def __init__(self, nodes, edges, state_qubit):
        self.edges = edges
        self.num_nodes = nodes
        self.state_qubit = state_qubit
        self.adjecency_matrix = self._set_adjecency_matrix(nodes, edges)
        self.H_X, self.H_Z = self.parity_check_matrix()

    def _set_adjecency_matrix(self, dimension, edges):
        adjecency_matrix = np.zeros((dimension, dimension))
        for e in edges:
            i, j = e
            adjecency_matrix[int(i)-1][int(j)-1] = 1
            adjecency_matrix[int(j)-1][int(i)-1] = 1
        return adjecency_matrix

    def parity_check_matrix(self):
        H_X = np.zeros((self.num_nodes, self.num_nodes))
        H_Z = np.zeros((self.num_nodes, self.num_nodes))
        H_Z = self.adjecency_matrix
        for i in range(len(H_X)):
            H_X[i][i] = 1
        return H_X, H_Z

    @staticmethod
    def stabilizer_generators(H_X, H_Z):
        dim = H_X.shape[-1]
        generators = []
        for i in range(len(H_X)):
            stabilizer = ["I"]*dim
            for j in range(dim):
                if H_Z[i][j] == 1:
                    stabilizer[j] = "Z"
                if H_X[i][j] == 1:
                    stabilizer[j] = "X"
            stabilizer = "".join(stabilizer)
            generators.append(stabilizer)
        return generators

    @staticmethod
    def offset(n, lst):
        if len(lst) == 0:
            return 0
        else:
            off = list(filter(lambda x: n > x, lst))
            count = len(off)
            return count

    @staticmethod
    def XOR(arr1, arr2):
        arr = arr1 != arr2
        arr = arr.astype(int)
        return arr

    def measure(self, measurement):
        updated_h_x = np.copy(self.H_X)
        updated_h_z = np.copy(self.H_Z)
        state = int(self.state_qubit)-1
        updated_h_x = np.delete(updated_h_x,state,0)
        updated_h_z = np.delete(updated_h_z,state,0)
        qubits_measured = []
        for q in measurement:
            p = int(q)-1
            anti_commute = []
            p1 = p - ClusterState.offset(p,qubits_measured)
            for i in range(len(updated_h_z)):
                if updated_h_z[i][p1] == 1:
                    anti_commute.append(i)
            i1 = anti_commute[0]
            for i in range(1,len(anti_commute)):
                i2 = anti_commute[i]
                x = ClusterState.XOR(updated_h_x[i1], updated_h_x[i2])
                z = ClusterState.XOR(updated_h_z[i1], updated_h_z[i2])
                for i in range(len(x)):
                    updated_h_x[i2][i] = x[i]
                    updated_h_z[i2][i] = z[i]
            qubits_measured.append(p1)
            h_x = np.delete(updated_h_x,i1,0)
            h_z = np.delete(updated_h_z,i1,0)
            h_x = np.delete(h_x,p1,1)
            h_z = np.delete(h_z,p1,1)
            updated_h_x = h_x
            updated_h_z = h_z
        return updated_h_x, updated_h_z

In [ ]:
nodes = 17

edges = [('1','11'),('1','12'),('1','13'),('2','11'),
     ('2','12'),('3','11'),('3','13'),('4','11'),
     ('4','14'),('4','15'),('5','11'),('5','14'),
     ('6','11'),('6','15'),('7','11'),('7','16'),
     ('7','17'), ('8','11'),('8','16'),('9','11'),
     ('9','17'),('10','11')]

cluster_state_shor = ClusterState(nodes,edges,'10')
sg = cluster_state_shor.stabilizer_generators(cluster_state_shor.H_X, cluster_state_shor.H_Z)
sg

['XIIIIIIIIIZZZIIII',
 'IXIIIIIIIIZZIIIII',
 'IIXIIIIIIIZIZIIII',
 'IIIXIIIIIIZIIZZII',
 'IIIIXIIIIIZIIZIII',
 'IIIIIXIIIIZIIIZII',
 'IIIIIIXIIIZIIIIZZ',
 'IIIIIIIXIIZIIIIZI',
 'IIIIIIIIXIZIIIIIZ',
 'IIIIIIIIIXZIIIIII',
 'ZZZZZZZZZZXIIIIII',
 'ZZIIIIIIIIIXIIIII',
 'ZIZIIIIIIIIIXIIII',
 'IIIZZIIIIIIIIXIII',
 'IIIZIZIIIIIIIIXII',
 'IIIIIIZZIIIIIIIXI',
 'IIIIIIZIZIIIIIIIX']

In [ ]:
measurement = ["10", "11", "12", "13", "14", "15", "16", "17"]
final_h_x, final_h_z = cluster_state_shor.measure(measurement)
ls = [final_h_x, final_h_z]
final_h_x, final_h_z

(array([[1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 1.]]))

In [ ]:
final_state = cluster_state_shor.stabilizer_generators(final_h_x, final_h_z)
final_state

['XXXXXXIII',
 'XXXIIIXXX',
 'ZZIIIIIII',
 'ZIZIIIIII',
 'IIIZZIIII',
 'IIIZIZIII',
 'IIIIIIZZI',
 'IIIIIIZIZ']

In [ ]:
cluster_state_3_1 = ClusterState(5,[('1','4'),('4','2'),('2','5'),('5','3')],'2')
code_3_1 = cluster_state_3_1.stabilizer_generators(cluster_state_3_1.H_X, cluster_state_3_1.H_Z)
code_3_1

['XIIZI', 'IXIZZ', 'IIXIZ', 'ZZIXI', 'IZZIX']

In [ ]:
h_x_3_1, h_z_3_1 = cluster_state_3_1.measure(["4","5"])
state_3_1 = cluster_state_3_1.stabilizer_generators(h_x_3_1, h_z_3_1)
state_3_1

['ZZI', 'IZZ']

In [ ]:
steane_edges = [("1","10"),("1","11"),("1","12"),("2","9"),
                ("2","11"),("2","12"),("3","9"),("3","10"),
                ("3","12"),("8","9"),("8","10"),("8","11"),
                ("9","4"),("10","5"),("11","6"),("12","7")]
cluster_state_steane = ClusterState(12,steane_edges,'8')
code_steane = cluster_state_steane.stabilizer_generators(cluster_state_steane.H_X, cluster_state_steane.H_Z)
code_steane

['XIIIIIIIIZZZ',
 'IXIIIIIIZIZZ',
 'IIXIIIIIZZIZ',
 'IIIXIIIIZIII',
 'IIIIXIIIIZII',
 'IIIIIXIIIIZI',
 'IIIIIIXIIIIZ',
 'IIIIIIIXZZZI',
 'IZZZIIIZXIII',
 'ZIZIZIIZIXII',
 'ZZIIIZIZIIXI',
 'ZZZIIIZIIIIX']

In [ ]:
h_x_steane, h_z_steane = cluster_state_steane.measure(["8","9","10","11","12"])
state_steane = cluster_state_steane.stabilizer_generators(h_x_steane, h_z_steane)
state_steane

['XXIXXII', 'XIXXIXI', 'XXXIIIX', 'ZZIZZII', 'ZIZZIZI', 'ZZZIIIZ']